In [74]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Disable Warnings
import warnings
warnings.filterwarnings('ignore')

# NLP functionalities and libraries
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Classification evaluation
from sklearn.metrics import (confusion_matrix, accuracy_score, precision_score, 
                             recall_score, f1_score)

# Word Cloud
from wordcloud import WordCloud, STOPWORDS

import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
# Others
import re
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

from sklearn.manifold import TSNE

from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup as soup
from nltk.stem.snowball import SnowballStemmer

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model, Model
from keras.utils import plot_model
from keras.layers import Flatten, Dropout, Activation, Input, Dense, concatenate, LSTM, Conv1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

RANDOM_SEED = 101

In [2]:
import spacy

In [3]:
# Importing the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [4]:
train_data.head()

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2
2,fe809672251f6bd0d986e00380f48d047c7e7b76,Interesting that it only targets S1P-1/5 recep...,fingolimod,2
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,"Very interesting, grand merci. Now I wonder wh...",ocrevus,2
4,b227688381f9b25e5b65109dd00f7f895e838249,"Hi everybody, My latest MRI results for Brain ...",gilenya,1


In [5]:
test_data.head()

,unique_hash,text,drug
0,9e9a8166b84114aca147bf409f6f956635034c08,"256 (previously stable on natalizumab), with 5...",fingolimod
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,On fingolimod and have been since December 201...,fingolimod
2,50b6d851bcff4f35afe354937949e9948975adf7,Apparently it's shingles! :-/ I do have a few ...,humira
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,If the Docetaxel doing once a week x3 weeks th...,tagrisso
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,"CC, Stelara worked in a matter of days for me....",stelara


In [6]:
train_data.shape

(5279, 4)

In [7]:
test_data.shape

(2924, 3)

In [8]:
train_data['drug'].nunique()

102

In [9]:
test_data['drug'].nunique()

95

In [10]:
len(set(train_data['drug'].unique()).intersection(set(test_data['drug'].unique())))

86

In [11]:
from sklearn.preprocessing import LabelEncoder
drug_le = LabelEncoder()

In [12]:
drug_le.fit(train_data['drug'])

LabelEncoder()

In [13]:
drug_dict = dict(zip(drug_le.classes_, drug_le.transform(drug_le.classes_)))
train_data['drug_enc'] = train_data['drug'].map(drug_dict)

In [14]:
test_data['drug_enc'] = test_data['drug'].map(drug_dict).fillna("Unknown")

In [15]:
train_dummy_matrix = np.zeros((train_data.shape[0],len(drug_le.classes_)))

In [16]:
test_dummy_matrix = np.zeros((train_data.shape[0],len(drug_le.classes_)))

In [17]:
for i in range(train_data.shape[0]):
    if train_data['drug'][i] in drug_dict:
        train_dummy_matrix[i][drug_dict[train_data['drug'][i]]] = 1    

In [18]:
for i in range(test_data.shape[0]):
    if test_data['drug'][i] in drug_dict:
        test_dummy_matrix[i][drug_dict[test_data['drug'][i]]] = 1  

In [19]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,:!./\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " is ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"(a-z)(\))", r"(a-z) ", text)
    text = re.sub(r"(\()(a-z)", r" (a-z)", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    ## Lemmatization
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    lemmed_words = [lemmatizer.lemmatize(word) for word in text]
    text = " ".join(lemmed_words)
    return text

In [20]:
# apply the above function to train
train_data['Clean_text'] = train_data['text'].map(lambda x: clean_text(x))

In [21]:
# apply the above function to train
test_data['Clean_text'] = test_data['text'].map(lambda x: clean_text(x))

In [22]:
test_data.head()

,unique_hash,text,drug,drug_enc,Clean_text
0,9e9a8166b84114aca147bf409f6f956635034c08,"256 (previously stable on natalizumab), with 5...",fingolimod,35,256 previously stable natalizumab 55 switching...
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,On fingolimod and have been since December 201...,fingolimod,35,fingolimod since december 2015; way describe b...
2,50b6d851bcff4f35afe354937949e9948975adf7,Apparently it's shingles! :-/ I do have a few ...,humira,43,apparently shingle ! : - / red spot left breas...
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,If the Docetaxel doing once a week x3 weeks th...,tagrisso,88,docetaxel week week week claim le harsh effica...
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,"CC, Stelara worked in a matter of days for me....",stelara,86,cc stelara worked matter day me willing jump h...


In [23]:
train_data.head()

,unique_hash,text,drug,sentiment,drug_enc,Clean_text
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2,38,autoimmune disease tend come cluster gilenya f...
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2,38,completely understand you d want try it but re...
2,fe809672251f6bd0d986e00380f48d047c7e7b76,Interesting that it only targets S1P-1/5 recep...,fingolimod,2,35,interesting target s1p - 1/5 receptor rather 1...
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,"Very interesting, grand merci. Now I wonder wh...",ocrevus,2,64,interesting grand merci wonder lemtrada ocrevu...
4,b227688381f9b25e5b65109dd00f7f895e838249,"Hi everybody, My latest MRI results for Brain ...",gilenya,1,38,everybody latest mri result brain cervical cor...


In [24]:
train_data['sentiment'].unique()

array([2, 1, 0])

In [25]:
dummy_y = pd.get_dummies(train_data['sentiment']).values
dummy_y[:10]

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]], dtype=uint8)

In [26]:
from sklearn.model_selection import train_test_split

In [30]:
text_train, text_test, y_train, y_test, drug_train, drug_test = train_test_split(train_data['Clean_text'].values,
                                                                                 dummy_y,
                                                                                 train_dummy_matrix,
                                                                                 test_size = 0.25,
                                                                                 random_state = RANDOM_SEED)

In [35]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_train)

In [46]:
vocabulary_size = len(tokenizer.word_index) + 1
MAXLEN = 100
embedding_size = 32
op_units = train_data['sentiment'].nunique()

In [44]:
trainsequences = tokenizer.texts_to_sequences(text_train)
trainseqs = pad_sequences(trainsequences, maxlen=MAXLEN, padding='post')
print(trainseqs)

[[20229  1373  3343 ...     0     0     0]
 [  615    27    55 ...     0     0     0]
 [  205  9892   406 ...     0     0     0]
 ...
 [  280   247   124 ...     0     0     0]
 [   15 12391   934 ...     0     0     0]
 [  386    25  7417 ...  3408   590   751]]


In [45]:
testsequences = tokenizer.texts_to_sequences(text_test)
testseqs = pad_sequences(testsequences, maxlen=MAXLEN, padding='post')

In [ ]:
def f1(y_true, y_pred):
    f1_score
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [70]:
# define the model
model = Sequential()
model.add(Embedding(vocabulary_size, 
                    embedding_size, 
                    input_length=MAXLEN))
model.add(Flatten())
model.add(Dense(op_units, activation='softmax'))

# compile the model
model.compile(optimizer='adamax', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [71]:
# fit the model
model.fit(trainseqs, 
          y_train, 
          epochs=10,
          batch_size=32,
          validation_data=(testseqs,y_test),
          verbose=1)

Train on 3959 samples, validate on 1320 samples
Epoch 1/10
3959/3959 [==============================] - 5s 1ms/step - loss: 0.8499 - acc: 0.7131 - val_loss: 0.7709 - val_acc: 0.7220
Epoch 2/10
3959/3959 [==============================] - 3s 789us/step - loss: 0.7155 - acc: 0.7254 - val_loss: 0.7550 - val_acc: 0.7220
Epoch 3/10
3959/3959 [==============================] - 3s 795us/step - loss: 0.6459 - acc: 0.7257 - val_loss: 0.7436 - val_acc: 0.7220
Epoch 4/10
3959/3959 [==============================] - 3s 784us/step - loss: 0.5611 - acc: 0.7477 - val_loss: 0.7365 - val_acc: 0.7265
Epoch 5/10
3959/3959 [==============================] - 3s 805us/step - loss: 0.4690 - acc: 0.8065 - val_loss: 0.7299 - val_acc: 0.7303
Epoch 6/10
3959/3959 [==============================] - 3s 804us/step - loss: 0.3802 - acc: 0.8626 - val_loss: 0.7353 - val_acc: 0.7288
Epoch 7/10
3959/3959 [==============================] - 3s 818us/step - loss: 0.3004 - acc: 0.9131 - val_loss: 0.7462 - val_acc: 0.7250
Ep

In [72]:
trained_embeddings = model.layers[0].get_weights()[0]

In [80]:
trained_embeddings.shape

(34001, 32)

In [84]:
valsequences = tokenizer.texts_to_sequences(test_data['Clean_text'].values)
valseqs = pad_sequences(valsequences, maxlen=MAXLEN, padding='post')

In [85]:
y_pred = model.predict_classes(valseqs)

In [94]:
test_data.head()

,unique_hash,text,drug,drug_enc,Clean_text
0,9e9a8166b84114aca147bf409f6f956635034c08,"256 (previously stable on natalizumab), with 5...",fingolimod,35,256 previously stable natalizumab 55 switching...
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,On fingolimod and have been since December 201...,fingolimod,35,fingolimod since december 2015; way describe b...
2,50b6d851bcff4f35afe354937949e9948975adf7,Apparently it's shingles! :-/ I do have a few ...,humira,43,apparently shingle ! : - / red spot left breas...
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,If the Docetaxel doing once a week x3 weeks th...,tagrisso,88,docetaxel week week week claim le harsh effica...
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,"CC, Stelara worked in a matter of days for me....",stelara,86,cc stelara worked matter day me willing jump h...


In [95]:
submission_df = pd.concat([test_data['unique_hash'], 
                          pd.DataFrame(y_pred, columns=['sentiment'])],
                         axis=1)

In [96]:
submission_df.to_csv(r'submission.csv', index=False)

In [97]:
submission_df.head()

,unique_hash,sentiment
0,9e9a8166b84114aca147bf409f6f956635034c08,2
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,2
2,50b6d851bcff4f35afe354937949e9948975adf7,2
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,2
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,2
